In [1]:
import pandas as pd
import fba_utils
from fba_utils import get_flux_bounds, convert_transcriptomics_to_enzyme_activity
import cobra
import numpy as np
import os 

In [2]:
os.getcwd()

'/home/connah/Documents/Repos/syn_bmca/notebooks/notebooks_for_generating_data'

## 1) collect input

In [3]:
# get the cobra model for S. elongatus
syn_model = cobra.io.read_sbml_model('../../models/syn_elong.xml')
[m.id for m in syn_model.metabolites if m.id.startswith('sucr') ]

Restricted license - for non-production use only - expires 2025-11-24


['sucr_c', 'sucr_e']

In [4]:
# get the transcriptomics data 
transcript_df = pd.read_csv('../../data/oxidative_stress_experiments/raw_data/Se_Oxidative_Stress_data/Counts_ESeq2median_norm.csv',index_col=0)
display(transcript_df)

,WT100_0A,WT100_0B,WT205_0A,WT205_0B,WT205_5A,WT205_5B,WT205_10A,WT205_10B,WT205_20A,WT205_20B,...,AD205_0B,AD205_0A,AD205_50A,AD205_50B,AD205_60A,AD205_60B,AD205_70B,AD205_70A,AD205_80A,AD205_80B
SYNPCC7942_RS00005,1025.485166,1077.143332,1379.066646,1273.216108,950.393967,853.162525,1053.881429,1073.677047,1081.611269,1190.176299,...,1588.698865,1450.805622,1599.533895,1470.770720,1501.430567,1592.170242,1474.559114,1523.822277,1409.233180,1457.548082
SYNPCC7942_RS00010,164.178660,170.956156,233.043071,232.849432,261.123095,253.168296,211.279332,277.409381,290.265904,258.973547,...,252.428823,247.321506,248.731347,253.870552,259.898940,302.697482,263.314127,264.767642,285.013452,272.580421
SYNPCC7942_RS00015,2784.722648,2752.035462,3720.464093,3785.237078,3140.534518,2994.117541,3481.078516,3543.391117,3740.616445,3747.953328,...,4408.489094,3829.066892,3707.053734,3382.143225,3586.231417,3784.181367,3396.752244,3561.218668,3507.248870,3467.828684
SYNPCC7942_RS00020,5397.689159,5636.771157,5620.450546,5386.507066,2987.624598,3147.774599,4218.040947,4487.353593,4702.519513,4825.724087,...,5770.803379,5048.008602,5888.236318,5572.563527,5858.944810,5921.577349,5489.674857,5773.249035,6462.064258,6519.215059
SYNPCC7942_RS00025,248.793815,248.663500,389.319013,360.171043,494.016666,519.507198,418.785819,412.261163,592.184818,641.372783,...,423.719811,390.856309,706.971022,541.310764,594.588942,635.016736,693.960136,762.380585,689.662181,672.365037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HTX97_RS00025,63.145638,56.188387,64.429555,82.586991,84.688571,79.023630,99.351591,87.332583,91.105357,94.773298,...,80.136134,118.140184,44.006315,75.531735,97.228380,71.277389,90.036444,81.683634,64.215994,71.173776
HTX97_RS00030,46.727772,50.210899,58.946189,51.616869,57.635278,58.536022,61.623138,42.381989,58.265054,49.590679,...,102.173571,85.016768,70.792768,59.795957,68.246844,75.905791,86.638842,130.506036,81.809417,90.860140
HTX97_RS00035,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.001702,2.208228,0.956659,0.000000,0.000000,0.000000,1.698801,0.000000,0.879671,2.271504
HTX97_RS00040,8.840389,7.172986,6.854208,16.058582,5.881151,13.170605,11.318536,6.421513,14.831105,9.918136,...,12.020420,12.145253,14.349885,5.245259,11.218659,12.959525,16.138608,15.022278,10.556054,18.172028


## 2) Calculate additional data

In [5]:
# calculate enzyme activity
# Run enzyme activity converter for all strains in transcriptomics_df
all_enzyme_activity_df = convert_transcriptomics_to_enzyme_activity(transcript_df, syn_model)
display(all_enzyme_activity_df)


,Reaction_ID,WT100_0A,WT100_0B,WT205_0A,WT205_0B,WT205_5A,WT205_5B,WT205_10A,WT205_10B,WT205_20A,...,AD205_0B,AD205_0A,AD205_50A,AD205_50B,AD205_60A,AD205_60B,AD205_70B,AD205_70A,AD205_80A,AD205_80B
EX_gln__L_e: gln__L_e -->,EX_gln__L_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_hco3_e: hco3_e <=>,EX_hco3_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_mn2_e: mn2_e <=>,EX_mn2_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EX_arg__L_e: arg__L_e -->,EX_arg__L_e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADPT: ade_c + prpp_c <=> amp_c + ppi_c,ADPT,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZDS: h_c + nadph_c + o2_c + zcarote_c --> 2.0 h2o_c + nadp_c + norsp_c,ZDS,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
RDXRr: nadh_c + 2.0 rdxo_c <=> h_c + nad_c + 2.0 rdxr_c,RDXRr,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
MPTSS: atp_c + h_c + moadcoo_c --> moadamp_c + ppi_c,MPTSS,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
NOR: h_c + nadph_c + norsp_c + o2_c --> 2.0 h2o_c + lyc_c + nadp_c,NOR,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf


In [6]:
# check the enzyme conversion
all_enzyme_activity_df['WT100_0A'].dropna().min()

5.05165106167238

In [7]:
### Cleanup enzyme activity df
clean_enzyme_activity_df = all_enzyme_activity_df.copy()

# Replace current index with Reaction_ID column
clean_enzyme_activity_df = clean_enzyme_activity_df.set_index('Reaction_ID')

# Drop rows with all 0s: 
clean_enzyme_activity_df = clean_enzyme_activity_df.loc[~(clean_enzyme_activity_df==0).all(axis=1)]

# Drop rows with all infs:
clean_enzyme_activity_df.replace([np.inf, -np.inf], np.nan, inplace=True)
clean_enzyme_activity_df.dropna(how='all', inplace=True)

clean_enzyme_activity_df

,WT100_0A,WT100_0B,WT205_0A,WT205_0B,WT205_5A,WT205_5B,WT205_10A,WT205_10B,WT205_20A,WT205_20B,...,AD205_0B,AD205_0A,AD205_50A,AD205_50B,AD205_60A,AD205_60B,AD205_70B,AD205_70A,AD205_80A,AD205_80B
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
DTMPK,75.774766,69.338860,72.654605,59.646160,105.860714,150.730257,103.124436,104.028518,112.292649,94.773298,...,78.132731,85.016768,116.712401,114.346654,119.665699,138.852056,153.741474,141.772744,139.867713,164.305420
ACOATA,503.902193,500.913492,640.183026,516.168692,544.594562,519.507198,692.945904,697.376360,786.048542,840.837515,...,1365.319390,601.742058,1972.630916,1507.487535,1604.268277,1879.131158,1640.192194,1668.411699,1519.192074,1847.489517
3HAD160,113.662149,117.158764,67.171238,89.469240,128.209087,73.170028,101.866821,136.136085,104.877097,119.017630,...,147.250147,126.973095,141.585536,86.022253,118.730811,115.710047,110.422053,92.011450,96.763826,94.645979
RNDR3,14528.548453,14460.738900,17168.420155,16882.157368,8550.017008,9452.104202,11162.591367,11448.274163,11464.443832,11599.810857,...,14606.813900,14203.320791,16527.241370,16963.168718,16584.917973,16487.293132,15442.948873,15532.096079,15263.174101,14831.403551
G5SD,2071.176935,2068.210838,1910.953186,1967.176239,1829.037893,1704.861650,1780.782940,1862.238898,1756.426527,1681.675030,...,2089.549705,2003.966669,1813.825517,1840.036978,1791.245932,1893.016364,1744.668444,1718.172993,1905.367708,1950.464343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MDDCP5pp,1422.039774,1558.928863,1273.511843,1409.714051,1145.648173,1049.258200,1123.050258,1004.324702,1081.611269,1053.526428,...,1132.924600,1034.554694,1056.151567,1100.455410,1089.144839,1015.471370,1166.226764,1143.570877,1179.639010,1225.097557
MDDEP2pp,2671.060499,2793.877878,2416.793735,2274.583371,1990.181425,2037.053576,1872.588840,1760.778986,2198.181570,2152.235474,...,2534.305251,1949.865090,3085.225366,3023.367486,2942.093397,3119.542859,3412.041451,3527.418544,3398.169647,3836.569420
MDDEP3pp,2671.060499,2793.877878,2416.793735,2274.583371,1990.181425,2037.053576,1872.588840,1760.778986,2198.181570,2152.235474,...,2534.305251,1949.865090,3085.225366,3023.367486,2942.093397,3119.542859,3412.041451,3527.418544,3398.169647,3836.569420


In [8]:
for i in clean_enzyme_activity_df.index:
    print(i)

DTMPK
ACOATA
3HAD160
RNDR3
G5SD
HISTDa
TRPS3
DMBZIDS2
HSTPT
ZCAROTDH2
ORNDC
GND
CMPA_1
U23GAAT2
HOXGfx
PGK
IMPD
LPOR
CHRPL
GCALDDy
MSBENZMT
DESAT18a
FUM
PHYFXOR
TMPPP_1
VPAMTr
RNDR1
TALA
GLYCL
ACONT
NDPK7
GTPCI
AIRC2
IGPS
MTAP
HEMEAS_1
DHORD3um
MTHFC
HBZNPT
GCATENEC
COCHL_1
HSDy
TRPS1
HISTDb
UPPDC1
GARFT
H4THDPR
UDPG4E
GLCS3
AICART
ASPTA
GLYOX
PRAGSr
SQD1
ACKr
UDCPDPS
GK1
SHKK
OXADC
G1PACT
3HAD60
PGI
KAS14
OHPBAT
3HAD80
3OAR160
NMNS
PPK
GALUi
PHYTES2_1
IGPDH
OMPDC
DCTPD
GLUSfx
NADS2
GLUTRR
G1PTT
UAMAGS
SPS
ADCL
CYTK1
DAPDC
PSIum
R05219
Cobalt2abcppI
PFK_3
THRS
NAMNPP
FOLD3
P5CR
MEHLER_1
ICHORS
3OAS140
PPK2
IPDPS_syn
GLYDHDA
SBP
H4THDPS
EAR60y
ZNabcpp
UAPGR
DHAD1
IPPS
ANS
SUCBZL
ADSL1r
NNATr
PERD
PPBNGS
ADOCBLS
LTHRK
PPS
PDS2_1
ALAALAr
ADCPS2
GF6PTA
PGMT
GLCP2_1
AGDI_1
LPADSS2
GLYCK
DPR
EAR120y
MAN1PT
NMNAT
TKT2
PSP_L
HEX7
PPNCL2
CHORS
CYTBD4cm
PPNCL3
CS
PPCDC
LDH_D
HPYRRy
ADMDC
PRE3BS
PPPGO2_1
BIOMASS_DNA
DMATT
DNAMTn2r
G3PD2
PHYTES
MI3PP
MOHMT
FBA
PROAKGOX1
ASAD
CYSS_2
LYCOPC
ACOTA
3H

## 3) Normalize dataframes with respect to reference condition

In [9]:
# define reference condition

# we only have transcriptomic data so use the first datapoint as the reference
ref_cond = 'WT100_0A'
display(ref_cond)

'WT100_0A'

In [10]:
# normalize transcriptomics data
norm_enzyme_activity_df = clean_enzyme_activity_df.div(clean_enzyme_activity_df[ref_cond],axis=0)
display(norm_enzyme_activity_df)
display(norm_enzyme_activity_df[ref_cond])



,WT100_0A,WT100_0B,WT205_0A,WT205_0B,WT205_5A,WT205_5B,WT205_10A,WT205_10B,WT205_20A,WT205_20B,...,AD205_0B,AD205_0A,AD205_50A,AD205_50B,AD205_60A,AD205_60B,AD205_70B,AD205_70A,AD205_80A,AD205_80B
Reaction_ID,,,,,,,,,,,,,,,,,,,,,
DTMPK,1.0,0.915065,0.958823,0.787151,1.397044,1.989188,1.360934,1.372865,1.481927,1.250724,...,1.031118,1.121967,1.540254,1.509033,1.579229,1.832431,2.028927,1.870976,1.845835,2.168340
ACOATA,1.0,0.994069,1.270451,1.024343,1.080754,1.030968,1.375160,1.383952,1.559923,1.668652,...,2.709493,1.194164,3.914710,2.991627,3.183690,3.729159,3.254981,3.310983,3.014855,3.666365
3HAD160,1.0,1.030763,0.590973,0.787151,1.127984,0.643750,0.896225,1.197726,0.922709,1.047118,...,1.295507,1.117110,1.245670,0.756824,1.044594,1.018017,0.971494,0.809517,0.851328,0.832696
RNDR3,1.0,0.995333,1.181702,1.161999,0.588498,0.650588,0.768321,0.787985,0.789098,0.798415,...,1.005387,0.977615,1.137570,1.167575,1.141540,1.134820,1.062938,1.069074,1.050564,1.020846
G5SD,1.0,0.998568,0.922641,0.949787,0.883091,0.823137,0.859793,0.899121,0.848033,0.811942,...,1.008871,0.967550,0.875746,0.888402,0.864844,0.913981,0.842356,0.829564,0.919944,0.941718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MDDCP5pp,1.0,1.096262,0.895553,0.991332,0.805637,0.737854,0.789746,0.706256,0.760605,0.740856,...,0.796690,0.727515,0.742702,0.773857,0.765903,0.714095,0.820108,0.804176,0.829540,0.861507
MDDEP2pp,1.0,1.045981,0.904807,0.851566,0.745090,0.762639,0.701066,0.659206,0.822962,0.805761,...,0.948801,0.729997,1.155056,1.131898,1.101470,1.167904,1.277411,1.320606,1.272217,1.436347
MDDEP3pp,1.0,1.045981,0.904807,0.851566,0.745090,0.762639,0.701066,0.659206,0.822962,0.805761,...,0.948801,0.729997,1.155056,1.131898,1.101470,1.167904,1.277411,1.320606,1.272217,1.436347


Reaction_ID
DTMPK       1.0
ACOATA      1.0
3HAD160     1.0
RNDR3       1.0
G5SD        1.0
           ... 
MDDCP5pp    1.0
MDDEP2pp    1.0
MDDEP3pp    1.0
MDDEP4pp    1.0
MNHNAtpp    1.0
Name: WT100_0A, Length: 682, dtype: float64

## 4) Save to processed data

In [11]:
norm_enzyme_activity_df_fname = '../../data/oxidative_stress_experiments/processed_data/normalized_enzyme_activity.csv'
enzyme_activity_df_fname = '../../data/oxidative_stress_experiments/processed_data/cleaned_enzyme_activity.csv'

norm_enzyme_activity_df.to_csv(norm_enzyme_activity_df_fname)

clean_enzyme_activity_df.to_csv(enzyme_activity_df_fname)
